In [3]:
import requests

# On demande la liste complète de tous les réseaux mondiaux
url = "https://api.citybik.es/v2/networks"
headers = {'User-Agent': 'Mozilla/5.0'}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    
    print(f"Analyse de {len(data['networks'])} réseaux...\n")
    
    trouve = False
    for network in data['networks']:
        # On cherche si la ville contient "Bordeaux"
        city_name = network['location']['city']
        
        if "Bordeaux" in city_name:
            print(f"✅ TROUVÉ !")
            print(f"Ville : {city_name}")
            print(f"Nom du réseau : {network['name']}")
            print(f"⚠️ NOUVEL ID À UTILISER : {network['id']}")
            print(f"Nouvelle URL : https://api.citybik.es/v2/networks/{network['id']}")
            trouve = True
            break
            
    if not trouve:
        print("❌ Bordeaux n'est plus listé sur CityBik.es pour le moment.")
        print("Il faut utiliser la source officielle (voir Solution 2 ci-dessous).")

except Exception as e:
    print(f"Erreur : {e}")

Analyse de 787 réseaux...

✅ TROUVÉ !
Ville : Bordeaux
Nom du réseau : V³
⚠️ NOUVEL ID À UTILISER : v3-bordeaux
Nouvelle URL : https://api.citybik.es/v2/networks/v3-bordeaux


In [23]:
import requests

url = "https://api.citybik.es/v2/networks/v3-bordeaux"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    recherche = "Quinconces" 
    
    print(f"🔍 Résultats détaillés pour '{recherche}' :")
    print("-" * 50)

    count = 0
    for station in data['network']['stations']:
        nom_station = station['name']
        
        if recherche.lower() in nom_station.lower():
            # 1. Récupération des données de base
            total_velos = station['free_bikes']
            places = station['empty_slots']
            
            # 2. Extraction des données cachées dans 'extra'
            # On utilise .get() pour éviter un crash si l'info est absente
            extra_data = station.get('extra', {})
            
            # CityBik.es stocke souvent les électriques sous 'ebikes'
            nb_elec = extra_data.get('ebikes', 0)
            
            # 3. Calcul des vélos classiques (Total - Électriques)
            # On s'assure que le résultat n'est pas négatif (sécurité)
            nb_classique = max(0, total_velos - nb_elec)

            # Affichage
            print(f"📍 {nom_station}")
            print(f"   ⚡ Vélos Électriques : {nb_elec}")
            print(f"   🚲 Vélos Classiques  : {nb_classique}")
            print(f"   🅿️ Places disponibles: {places}")
            print(f"   (Total vélos sur place : {total_velos})")
            print("-" * 50)
            count += 1

    if count == 0:
        print("Aucune station trouvée.")

except Exception as e:
    print(f"Une erreur est survenue : {e}")

🔍 Résultats détaillés pour 'Quinconces' :
--------------------------------------------------
📍 Quinconces
   ⚡ Vélos Électriques : 2
   🚲 Vélos Classiques  : 13
   🅿️ Places disponibles: 50
   (Total vélos sur place : 15)
--------------------------------------------------


In [5]:
import requests
import json
import unicodedata
from datetime import datetime

# --- CONFIGURATION ---
API_KEY = "PWX0HJQMSI"
# Dataset TBM temps réel
URL = f"https://data.bordeaux-metropole.fr/geojson?key={API_KEY}&typename=ci_vcub_p"

def remove_accents(input_str):
    """Retire les accents pour faciliter la recherche (ex: Fauré -> Faure)"""
    if not isinstance(input_str, str):
        return ""
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def formater_date(date_str):
    """Rend la date plus lisible (enlève le T et les millisecondes)"""
    try:
        # Format TBM souvent ISO : 2023-10-27T10:00:00...
        if not date_str: return "?"
        # On garde juste l'heure pour l'affichage simple
        dt = datetime.fromisoformat(date_str)
        return dt.strftime("%H:%M:%S")
    except:
        return date_str

def verifier_stations(recherche=None):
    print(f"🔌 Connexion à l'API Bordeaux Métropole...")
    
    try:
        response = requests.get(URL)
        
        if response.status_code != 200:
            print(f"❌ Erreur technique : {response.status_code}")
            return

        data = response.json()
        features = data.get('features', [])
        
        print(f"✅ Données reçues ! {len(features)} stations en mémoire.\n")
        
        # Si aucune recherche n'est spécifiée, on liste les 10 premières
        if not recherche:
            print("Liste de quelques stations disponibles :")
            for i, s in enumerate(features[:10]):
                print(f"- {s['properties'].get('nom')}")
            return

        print(f"🔍 Recherche pour : '{recherche}'")
        print("-" * 40)
        
        count = 0
        recherche_simple = remove_accents(recherche.lower())

        for station in features:
            props = station['properties']
            
            # --- CORRECTION : UTILISATION DES CLÉS EN MINUSCULES ---
            # Grâce à votre log, on sait que les clés sont : 'nom', 'nbvelos', 'nbelec', etc.
            
            nom_reel = props.get('nom', 'Inconnu')
            nom_clean = remove_accents(nom_reel.lower())
            
            if recherche_simple in nom_clean:
                # On utilise les données précises fournies par votre clé API
                velos_total = props.get('nbvelos', 0)
                velos_elec = props.get('nbelec', 0)      # Donnée native !
                velos_classique = props.get('nbclassiq', 0) # Donnée native !
                places = props.get('nbplaces', 0)
                etat = props.get('etat', 'INCONNU')
                heure_maj = formater_date(props.get('mdate'))
                
                print(f"📍 {nom_reel}")
                print(f"   🚲 Total Vélos : {velos_total}")
                print(f"      ⚡ Électriques : {velos_elec}")
                print(f"      🚲 Classiques  : {velos_classique}")
                print(f"   🅿️ Places libres : {places}")
                print(f"   📡 État : {etat} (MAJ: {heure_maj})")
                print("-" * 40)
                count += 1
        
        if count == 0:
            print("❌ Aucune station trouvée.")
            
    except Exception as e:
        print(f"⚠️ Erreur : {e}")

if __name__ == "__main__":
    # Test avec "Faure"
    verifier_stations("Faure")

🔌 Connexion à l'API Bordeaux Métropole...
✅ Données reçues ! 203 stations en mémoire.

🔍 Recherche pour : 'Faure'
----------------------------------------
📍 Faure-Brunet
   🚲 Total Vélos : 11
      ⚡ Électriques : 6
      🚲 Classiques  : 5
   🅿️ Places libres : 8
   📡 État : CONNECTEE (MAJ: 16:07:00)
----------------------------------------
